In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import xgboost as xgb
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import LabelEncoder,OneHotEncoder,MinMaxScaler,StandardScaler

from sklearn.metrics import r2_score

import matplotlib.pyplot as plt
import seaborn as sbn

data=pd.read_csv('data.csv',encoding='gbk')

real_data=pd.read_csv('sub_a_913.csv')

In [2]:

#根据分析，可以将数据属性分为类别特征和连续型特征
#类别特征
category_feauture=[ 'rentType', 'houseType', 'houseFloor','houseToward', 'houseDecoration', 'communityName', 'region',
                    'plate', 'buildYear', 'room_lobby', 'room_toilet', 'lobby_toilet']

#连续型特征
Value_feauture=[f for f in data.columns if f not in category_feauture and f not in ['ID','tradeTime','tradeMoney','type','city']]
print(Value_feauture)

target=['tradeMoney']

['area', 'totalFloor', 'saleSecHouseNum', 'totalTradeMoney', 'totalTradeArea', 'tradeMeanPrice', 'tradeSecNum', 'totalNewTradeMoney', 'totalNewTradeArea', 'tradeNewMeanPrice', 'tradeNewNum', 'remainNewNum', 'supplyNewNum', 'supplyLandNum', 'supplyLandArea', 'tradeLandNum', 'tradeLandArea', 'landTotalPrice', 'landMeanPrice', 'totalWorkers', 'newWorkers', 'residentPopulation', 'pv', 'uv', 'lookNum', 'month', 'days', 'room_num', 'lobby_num', 'toilet_num', 'transport', 'school', 'hospital', 'mall', 'other', 'communityName_count', 'totalFloor_count', 'buildYear_count', 'plate_count', 'totalFloorcommunityName_count', 'buildYearcommunityName_count', 'plateregion_count', 'ave_money', 'community_mean_area', 'community_std_area', 'community_mean_ave_money', 'community_std_ave_money', 'plate_mean_ave_money_x', 'plate_std_ave_money_x', 'plate_mean_area', 'plate_std_area', 'plate_mean_ave_money_y', 'plate_std_ave_money_y', 'community_mean_transport', 'community_std_transport', 'community_mean_mall'

In [3]:
for f in category_feauture:
    ln=LabelEncoder()
    data[f]=ln.fit_transform(data[f])
'''   
for f in Value_feauture:
    if f not in ['ave_money']:
        mms=MinMaxScaler(feature_range=(0,1))
        data[f]=mms.fit_transform(data[f].values.reshape(-1,1))''' 

feas=category_feauture+Value_feauture

In [4]:
'''from sklearn.mixture import GaussianMixture
def cluster(data):
    cols=[ 'rentType', 'houseType', 'houseFloor','houseToward', 'houseDecoration', 'communityName', 'region',
          
         'plate', 'buildYear', 'room_lobby', 'room_toilet', 'lobby_toilet', 
          
        'totalFloor', 'saleSecHouseNum', 'totalTradeMoney', 'totalTradeArea', 'tradeMeanPrice', 'tradeSecNum', 
          
          'totalNewTradeMoney', 'totalNewTradeArea', 'tradeNewMeanPrice', 'tradeNewNum', 'remainNewNum', 'supplyNewNum',
          
          'supplyLandNum', 'supplyLandArea', 'tradeLandNum', 'tradeLandArea', 'landTotalPrice',
          
          'landMeanPrice', 'totalWorkers', 'newWorkers', 'residentPopulation', 'pv', 'uv', 'lookNum',
          
          'month', 'days', 'room_num', 'lobby_num', 'toilet_num', 'transport', 'school', 'hospital', 'mall',
          
          'other', 'communityName_count', 'totalFloor_count','buildYear_count', 'plate_count',
          
          'totalFloorcommunityName_count', 'buildYearcommunityName_count', 'plateregion_count']
    
    gmm=GaussianMixture(n_components=3, covariance_type='full', random_state=0)
    data['cluster']=pd.DataFrame(gmm.fit(data[cols]).predict(data[cols]))
    
    col1=['rentType', 'houseType', 'houseFloor','houseToward', 'houseDecoration', 'communityName', 'region',
          
         'plate', 'buildYear', 'room_lobby', 'room_toilet', 'lobby_toilet']
    
    col2=['totalFloor', 'saleSecHouseNum', 'totalTradeMoney', 'totalTradeArea', 'tradeMeanPrice', 'tradeSecNum', 
          
          'totalNewTradeMoney', 'totalNewTradeArea', 'tradeNewMeanPrice', 'tradeNewNum', 'remainNewNum', 'supplyNewNum',
          
          'supplyLandNum', 'supplyLandArea', 'tradeLandNum', 'tradeLandArea', 'landTotalPrice',
          
          'landMeanPrice', 'totalWorkers', 'newWorkers', 'residentPopulation', 'pv', 'uv', 'lookNum',
          
          'month', 'days', 'room_num', 'lobby_num', 'toilet_num', 'transport', 'school', 'hospital', 'mall',
          
          'other', 'communityName_count', 'totalFloor_count','buildYear_count', 'plate_count',
          
          'totalFloorcommunityName_count', 'buildYearcommunityName_count', 'plateregion_count']
    
    for f1 in col1:
        for f2 in col2:
            temp=data.groupby(['cluster',f1])[f2].agg('mean').reset_index(name=f1+'feature'+f2)
            temp.fillna(0,inplace=True)
            data=data.merge(temp,on=['cluster',f1],how='left')
            
    return data

data=cluster(data)
#连续型特征
Value_feauture=[f for f in data.columns if f not in category_feauture and f not in ['ID','tradeTime','tradeMoney','type','city']]

print(Value_feauture)
'''
#做log平滑，与LabelEncoder 相比，log平滑线下下降，但线上有提高

'''for f in Value_feauture:
    if f not in ['ave_money']:
        mms=MinMaxScaler(feature_range=(0,1))
        data[f]=mms.fit_transform(data[f].values.reshape(-1,1))
'''
for f in Value_feauture:
    data[f]=data[f].map(lambda x:np.log1p(x))
    
feas=category_feauture+Value_feauture

In [5]:
#划分数据集

train_=data[data['type']=='train'][feas]
test_=data[data['type']=='test'][feas]
train_target=data[data['type']=='train']['tradeMoney']
test_target=data[data['type']=='test']['tradeMoney']

#划分数据为训练集和验证集，比例为 8:2

#train_x,valid_x,train_y,valid_y=train_test_split(train_,train_target,test_size=0.2,random_state=42)

In [ ]:
#构建baseline模型
from sklearn.model_selection import KFold
import hyperopt
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
import colorama
N_HYPEROPT_PROBES = 500
HYPEROPT_ALGO = tpe.suggest  #  tpe.suggest OR hyperopt.rand.suggest
obj_call_count = 0
cur_best_score = 0 # 0 or np.inf

space ={
        'num_leaves': hp.quniform ('num_leaves', 10, 100, 1),
        'min_data_in_leaf':  hp.quniform ('min_data_in_leaf', 10, 100, 1),
        'feature_fraction': hp.uniform('feature_fraction', 0.75, 1.0),
        'bagging_fraction': hp.uniform('bagging_fraction', 0.75, 1.0),
        'learning_rate': hp.uniform('learning_rate', 0, 0.01),
#         'learning_rate': hp.loguniform('learning_rate', -5.0, -2.3),
        'min_sum_hessian_in_leaf': hp.loguniform('min_sum_hessian_in_leaf', 0, 2.3),
        'max_bin': hp.quniform ('max_bin', 88, 200, 1),
        'bagging_freq': hp.quniform ('bagging_freq', 1, 15, 1),
        'lambda_l1': hp.uniform('lambda_l1', 0, 10 ),
        'lambda_l2': hp.uniform('lambda_l2', 0, 10 ),
       }

def get_lgb_params(space):
    lgb_params = dict()
    lgb_params['boosting_type'] = space['boosting_type'] if 'boosting_type' in space else 'gbdt'
    lgb_params['objective'] = 'regression'
    lgb_params['metric'] = 'rmse'
    lgb_params['learning_rate'] = space['learning_rate']
    lgb_params['num_leaves'] = int(space['num_leaves'])
    lgb_params['min_data_in_leaf'] = int(space['min_data_in_leaf'])
    lgb_params['min_sum_hessian_in_leaf'] = space['min_sum_hessian_in_leaf']
    lgb_params['max_depth'] = -1
    lgb_params['lambda_l1'] = space['lambda_l1'] if 'lambda_l1' in space else 0.0
    lgb_params['lambda_l2'] = space['lambda_l2'] if 'lambda_l2' in space else 0.0
    lgb_params['max_bin'] = int(space['max_bin']) if 'max_bin' in space else 256
    lgb_params['feature_fraction'] = space['feature_fraction']
    lgb_params['bagging_fraction'] = space['bagging_fraction']
    lgb_params['bagging_freq'] = int(space['bagging_freq']) if 'bagging_freq' in space else 1
    lgb_params['nthread'] = 4
    return lgb_params

def objective(space):
    global obj_call_count, cur_best_score

    obj_call_count += 1

    print('\nLightGBM objective call #{} cur_best_score={:7.5f}'.format(obj_call_count,cur_best_score) )
    params=get_lgb_params(space)

    kf=KFold(n_splits=5, shuffle=True, random_state=0)

    out_of_fold=np.zeros(len(train_))
    for fold,(train_idx,valid_idx) in enumerate(kf.split(train_)):
        lgb_train=lgb.Dataset(train_.iloc[train_idx],train_target[train_idx])
        lgb_eval=lgb.Dataset(train_.iloc[valid_idx],train_target[valid_idx],reference=lgb_train)

        gbm=lgb.train(params,lgb_train,num_boost_round=10000,valid_sets=[lgb_train,lgb_eval],
                    valid_names=['train','valid'],
                    early_stopping_rounds=200,
                    verbose_eval=1000,
                    )
        # predict
        nb_trees = gbm.best_iteration
        val_loss = gbm.best_score['valid_0']
        print('nb_trees={} val_loss={}'.format(nb_trees, val_loss))
        out_of_fold[valid_idx]=gbm.predict(train_.iloc[valid_idx],num_iteration=nb_trees)
        score=r2_score(out_of_fold,train_target)
        print(score)
    print("最终:",score)
    if score>cur_best_score:
        cur_best_score = score
        print(colorama.Fore.GREEN + 'NEW BEST SCORE={}'.format(cur_best_score) + colorama.Fore.RESET)
    return {'loss': -score, 'status': STATUS_OK}
    
trials = Trials()
best = hyperopt.fmin(fn=objective,
                     space=space,
                     algo=HYPEROPT_ALGO,
                     max_evals=N_HYPEROPT_PROBES,
                     trials=trials,
                     verbose=1)

print('-'*50)
print('The best params:')
print( best )
print('\n\n')

feature_rank=gbm.feature_importance()
features_df=pd.DataFrame({ 'column':feas,'importance': feature_rank}).sort_values(by='importance', ascending=False)
print(features_df)

                                                                                                                       
LightGBM objective call #1 cur_best_score=0.00000
Train until valid scores didn't improve in 200 rounds.                                                                 
[1000]	train's rmse: 1623.09	valid's rmse: 1839.56                                                                     
[2000]	train's rmse: 1377.7	valid's rmse: 1608.51                                                                      
[3000]	train's rmse: 1288.38	valid's rmse: 1556.99                                                                     
[4000]	train's rmse: 1224.08	valid's rmse: 1526.57                                                                     
[5000]	train's rmse: 1170.38	valid's rmse: 1503.82                                                                     
[6000]	train's rmse: 1126.65	valid's rmse: 1486.36                                                            

[3000]	train's rmse: 1257.61	valid's rmse: 1506.46                                                                     
[4000]	train's rmse: 1195.4	valid's rmse: 1471.8                                                                       
[5000]	train's rmse: 1147.52	valid's rmse: 1448.15                                                                     
[6000]	train's rmse: 1106.89	valid's rmse: 1430.16                                                                     
[7000]	train's rmse: 1070.05	valid's rmse: 1416.11                                                                     
[8000]	train's rmse: 1038.85	valid's rmse: 1405.79                                                                     
[9000]	train's rmse: 1010.34	valid's rmse: 1400.19                                                                     
[10000]	train's rmse: 984.379	valid's rmse: 1394.38                                                                    
nb_trees=0 val_loss={}                  

Early stopping, best iteration is:                                                                                     
[8514]	train's rmse: 887.627	valid's rmse: 1382.38
nb_trees=8514 val_loss={}                                                                                              
-3.6231395455154916                                                                                                    
Train until valid scores didn't improve in 200 rounds.                                                                 
[1000]	train's rmse: 1343.55	valid's rmse: 1624.28                                                                     
[2000]	train's rmse: 1197.19	valid's rmse: 1592.33                                                                     
[3000]	train's rmse: 1103.64	valid's rmse: 1577.34                                                                     
[4000]	train's rmse: 1032.03	valid's rmse: 1570.84                                                           

[5000]	train's rmse: 1084.92	valid's rmse: 1282.52                                                                     
Early stopping, best iteration is:                                                                                     
[5038]	train's rmse: 1082.91	valid's rmse: 1282.18
nb_trees=5038 val_loss={}                                                                                              
-0.2690835839201364                                                                                                    
Train until valid scores didn't improve in 200 rounds.                                                                 
[1000]	train's rmse: 1454.55	valid's rmse: 1655.04                                                                     
[2000]	train's rmse: 1315.42	valid's rmse: 1553.96                                                                     
[3000]	train's rmse: 1226.85	valid's rmse: 1499.8                                                            

                                                                                                                       
LightGBM objective call #6 cur_best_score=0.85961
Train until valid scores didn't improve in 200 rounds.                                                                 
[1000]	train's rmse: 1425.41	valid's rmse: 1634.16                                                                     
[2000]	train's rmse: 1280.91	valid's rmse: 1540.17                                                                     
[3000]	train's rmse: 1192.84	valid's rmse: 1493.54                                                                     
[4000]	train's rmse: 1125.29	valid's rmse: 1458.88                                                                     
[5000]	train's rmse: 1070.69	valid's rmse: 1437.05                                                                     
[6000]	train's rmse: 1025.14	valid's rmse: 1419.53                                                            

...

nb_trees=5204 val_loss={}                                                                                              
-0.2652664224072274                                                                                                    
Train until valid scores didn't improve in 200 rounds.                                                                 
[1000]	train's rmse: 1423.58	valid's rmse: 1595.53                                                                     
[2000]	train's rmse: 1281.96	valid's rmse: 1493.8                                                                      
[3000]	train's rmse: 1191.66	valid's rmse: 1438.76                                                                     
[4000]	train's rmse: 1125.29	valid's rmse: 1405.44                                                                     
[5000]	train's rmse: 1067.76	valid's rmse: 1379.64                                                                     
[6000]	train's rmse: 1019.05	valid's rms

[2000]	train's rmse: 1473.09	valid's rmse: 1567.71                                                                     
[3000]	train's rmse: 1395.54	valid's rmse: 1499.4                                                                      
[4000]	train's rmse: 1344.12	valid's rmse: 1459.66                                                                     
[5000]	train's rmse: 1304.19	valid's rmse: 1431.56                                                                     
[6000]	train's rmse: 1270.95	valid's rmse: 1407.71                                                                     
[7000]	train's rmse: 1240.78	valid's rmse: 1389.17                                                                     
[8000]	train's rmse: 1213.97	valid's rmse: 1371.73                                                                     
[9000]	train's rmse: 1190.86	valid's rmse: 1359.29                                                                     
[10000]	train's rmse: 1170.26	valid's rm

[6000]	train's rmse: 783.938	valid's rmse: 1361.87                                                                     
Early stopping, best iteration is:                                                                                     
[6040]	train's rmse: 782.2	valid's rmse: 1361.49
nb_trees=6040 val_loss={}                                                                                              
-3.6175846827791274                                                                                                    
Train until valid scores didn't improve in 200 rounds.                                                                 
[1000]	train's rmse: 1198.66	valid's rmse: 1573.33                                                                     
[2000]	train's rmse: 1017.59	valid's rmse: 1539.06                                                                     
[3000]	train's rmse: 911.428	valid's rmse: 1528.26                                                             

[1000]	train's rmse: 1148.94	valid's rmse: 1282.08                                                                     
[2000]	train's rmse: 972.221	valid's rmse: 1216.7                                                                      
[3000]	train's rmse: 872.005	valid's rmse: 1200.56                                                                     
Early stopping, best iteration is:                                                                                     
[3550]	train's rmse: 828.163	valid's rmse: 1198.4
nb_trees=3550 val_loss={}                                                                                              
0.3821934955990278                                                                                                     
Train until valid scores didn't improve in 200 rounds.                                                                 
[1000]	train's rmse: 1057.35	valid's rmse: 1648.74                                                            

[10000]	train's rmse: 1366.53	valid's rmse: 1531.24                                                                    
nb_trees=0 val_loss={}                                                                                                 
0.3129953829262516                                                                                                     
Train until valid scores didn't improve in 200 rounds.                                                                 
[1000]	train's rmse: 2375	valid's rmse: 2399.21                                                                        
[2000]	train's rmse: 1819.44	valid's rmse: 1895.72                                                                     
[3000]	train's rmse: 1622.58	valid's rmse: 1746.53                                                                     
[4000]	train's rmse: 1537.76	valid's rmse: 1693.31                                                                     
[5000]	train's rmse: 1487.22	valid's rms

In [ ]:
from sklearn.metrics import r2_score
r2=r2_score(valid,valid_y)
print("验证集结果:{}".format(r2))

# 未做特征工程前，验证集结果 :0.8480048499945494

#做特征工程，没加方差和均值统计特征前，验证集结果:0.8569968502829308（线下有提高）

#加了方差和均值统计特征，验证集结果:0.8669673740210536（有提高）

#做了聚类特征 验证集结果:0.8654237103479906，线下有点下降

In [ ]:
#训练整个数据模型，并预测
lgb_train_=lgb.Dataset(train_,train_target)
gbm=lgb.train(params,lgb_train_,num_boost_round=10000,valid_sets=[lgb_train_],
              valid_names=['train'],
              early_stopping_rounds=2000,
              verbose_eval=1000)

real_data['pred']=gbm.predict(test_)

#  测试结果记录

#未做很多的特征工程，只是补缺失值，然后将类别特征进行了处理得到结果：  0.40585301363216797

#删除异常数据后，进行预测结果：0.9211771459530279 ，这只是清除了异常数据就这么高???????是不是有点假??????

#做了归一化之后 结果是：0.9214346671924497

#做了特征工程后，没加方差和均值统计特征前：0.9433749685395988（有提高）

#加方差和均值统计特征 0.94636263988483（有些许提高）

#加了聚类特征，0.9279984686417421(线上也降低了)

In [ ]:
r2=r2_score(real_data['pred'],real_data['pre'])
print(r2)